In [1]:
# install.packages("pacman")
library(pacman)
pacman::p_load(gghighlight, ade4,  adegenet,  animation,  ape,  car,  caTools,  cowplot,  dartR,  devtools,  diveRsity,  dplyr, stringr, gganimate,  ggh4x,  ggplot2,  ggrepel,  grid,  gridExtra,  LEA,  magick,  mapplots,  OptGenMix,  oz,  ozmaps,  plyr,  poppr,  readxl,  reshape2,  rgl,  RRtools,  sfsCalcs,  tidyr,  tiff,  vegan,  webshot2, geosphere, HWxtest, phangorn, forcats, phytools) 
load("/home/richard/Packages/PSFsaved_scripts4autom.R")
maindir<- "/home/richard/rrspecies/"
RandRbase <- ""
setwd(maindir)
spls <- read.csv(paste0(maindir,"RRSpeciesParameters.csv"),header = TRUE)
nsp <- length(spls$species) #number of species


#for (z in 1:nsp){
z=1 # change this to the desired row you want to analyse from your RRParameters file
    
    species <- spls$species[z]
    dataset <- spls$dataset[z]
    analysis <- spls$analysis[z]
    topskip   <- spls$topskip[z]
    nmetavar   <- spls$nmetavar[z]
    max_missing <- spls$max_missing[z] #default is 0.2
    OptSiteMixMinSites <- spls$Run_OptSiteMix_minimum_Sites[z]
    OptSiteMixMinSamples <- spls$Run_OptSiteMix_minimum_samples[z]
    ForcedOptSiteMix <- spls$Run_forced_OptSiteMix[z]
    ListForcedSites <- spls$List_of_ForcedSites[z]
    thresh_maf <- spls$threshold_maf[z] #deafult 0.03 change to higher if singleton is stronger possibility e.g Cattai
    samplingthreshold <- spls$threshold_allele_prop[z] # deafult 0.9 the threshold value would want all sampling combinations to be over for allele prop script 

    num_steps <- 2000   #default of 2000 steps    
    rand_numsteps <- 100 #deafult of 100 randomisations

    print(paste0("Starting ",species))

    d1        <- read.dart.xls.onerow(RandRbase,species,dataset,topskip, nmetavar, euchits=FALSE)
    qc1       <- report.dart.qc.stats(d1, RandRbase, species, dataset, threshold_missing_loci = 0.8) # threshold_missing_loci: check which samples have 80% missing loci
    d2        <- remove.poor.quality.snps(d1, min_repro=0.96, max_missing=max_missing) 
    qc2       <- report.dart.qc.stats(d2, RandRbase, species, dataset)
    d3        <- sample.one.snp.per.locus.random(d2, seed=12345)
    qc3       <- report.dart.qc.stats(d3, RandRbase, species, dataset)
    m1        <- read.meta.data(d3, RandRbase, species, dataset, fields=10); colnames(m1$analyses)

    outputloc <- paste0(maindir, species, "/output_", analysis)
    outputloc2 <- paste0(maindir, species, "/output_", analysis,"/")

    subDir <-paste0(outputloc2)
    if(!dir.exists(file.path(subDir))){
        dir.create(file.path(subDir))
    }
     



   #create a temp meta dataframe to remove Sites/samples that include less than X individuals
    temp<-as.data.frame(m1$analyses)

    samplethreshold <-5                     # change this to what number of samples you want toi have uniform across all pops


  
#### Below lines remove Sites with less than threshold number of samples, and subsample Sites that are above the threshold

    for (x in unique(na.omit(temp[,analysis]))) {
        pop_samples <- which(temp[,analysis] == x)
        if (length(pop_samples) < samplethreshold) {
            temp[, analysis]<-gsub(x, replacement=NA, temp[, analysis])
            #print(paste0("Removing ",x, " with ", " with a sample size of ",length(pop_samples)," samples"))
        }
    
        if (length(pop_samples) == samplethreshold) {
        #print(paste0("Keeping ",x, " with ", " with a sample size of ",length(pop_samples)," samples"))
    
        }
    
        if (length(pop_samples) > samplethreshold) {
      
            subsamplepops <- pop_samples[sample(1:length(pop_samples), size=samplethreshold, replace = F)]
      
            for (b in 1:length(pop_samples)){
                if (pop_samples[b] %in% subsamplepops == FALSE){ 
                    temp[, analysis][pop_samples[b]] <- NA
                }
                #print(paste0("Reducing ",x, " to a sample size of ", samplethreshold," samples"))
            }
    
        } else {}
    }


### If you want to just remove any sites that are below the threshold (but keep individuals that are above a threshold, then unhash the below section)
# for (x in unique(na.omit(temp[, analysis]))) {
#   pop_samples <- which(temp[, analysis] == x)
#   if (length(pop_samples) < samplethreshold) {
#     temp[, analysis]<-gsub(x, replacement=NA, temp[, analysis])
#     print(paste0("Removing ",x, " with ", " with a sample size of ",length(pop_samples)," samples"))
#   } else {}
# }


    
    m1$analyses<-temp
    dm <- dart.meta.data.merge(d3, m1)
    fields    <- c(analysis) 
    dms     <- data.by.meta.fields(dm, fields, RandRbase, species, dataset, object=analysis); #print(dms$meta$analyses[,analysis])
    #correct inconsistencies in J's scripts for naming treatment which is for naming files
    treatment <- paste0("raw_SNPFilt_1SNPperClone_Field_", analysis)
    dms$treatment <- treatment  
    gta <- dms$gt 













    
#             █████╗ ██╗     ██╗     ███████╗██╗     ███████╗    ██████╗ ██████╗  ██████╗ ██████╗ 
#            ██╔══██╗██║     ██║     ██╔════╝██║     ██╔════╝    ██╔══██╗██╔══██╗██╔═══██╗██╔══██╗
#            ███████║██║     ██║     █████╗  ██║     █████╗      ██████╔╝██████╔╝██║   ██║██████╔╝
#            ██╔══██║██║     ██║     ██╔══╝  ██║     ██╔══╝      ██╔═══╝ ██╔══██╗██║   ██║██╔═══╝ 
#            ██║  ██║███████╗███████╗███████╗███████╗███████╗    ██║     ██║  ██║╚██████╔╝██║     
#            ╚═╝  ╚═╝╚══════╝╚══════╝╚══════╝╚══════╝╚══════╝    ╚═╝     ╚═╝  ╚═╝ ╚═════╝ ╚═╝     
                                                                                        
  
  
    ##### Allele proportion vs Number of Sites/Total Samples #####
    
 
  
    ###steps here are to remove missingness ###can be ignored if high missingness
    ###if ignored, make sure MUST do testing on amount of missing data (make sure 10,20,30% cutoff yields similar optgenmix results)
    ###if you want to remove all missingness (recommended sometimes use the first hashed out section. otherwise use the second)
    #cones GT matrix in new variable (ColID are SNPIDs)
    gt_sw_comp <- gta    
    ncol(gt_sw_comp)    
    #removes columns with ANY missingness
    # gt_sw_comp <- gt_sw_comp[  , which(colSums(gt_sw_comp, na.rm = ) > 0 & colSums(gt_sw_comp) < nrow(gt_sw_comp)*2) ] ###unhash this no missingness
    # ncol(gt_sw_comp)  ###unhash this no missingness
    subDir <-paste0(outputloc2, "OptSiteMix/")
  
    if(!dir.exists(file.path(subDir))){
        dir.create(file.path(subDir))
    }
  
    subDir <-paste0(outputloc2, "OptSiteMix/Allele_Proportion/")
    if(!dir.exists(file.path(subDir))){
        dir.create(file.path(subDir))
    }
  

    ###load required code
    source("/home/richard/Packages/common_allele_count.r")
    source("/home/richard/Packages/get_maf.r")
    # source("C:/Users/DimonR/OneDrive - DPIE/R/Packages/guide_axis_nested.r")

    ###Create a table with all possible combinations of sampling assuming all included pops have atleast 5 samples available
    sampling<-setNames(data.frame(matrix(ncol = 3, nrow = 0)), c("Sites", "samples", "total"))
    for (c in 1:length(unique(dms$meta$analyses[,analysis]))) {
        for (ind in 1:samplethreshold) {
            sampling[nrow(sampling) + 1,] = list(c,ind,c*ind)
        }
    }
  
    ###this bit orders the sampling strategy and removes strategies with less than x samples
    sampling<-sampling[order(sampling$total),]
    # sampling<-sampling[which(sampling$total>=samplethreshold),] #this reduces sampling to 5 samples per site
    sampling<-sampling[which(sampling$Sites<=20),] # this reduces sampling to 20 Sites.

    ###N_t_vec is the total number of individuals to be sampled (i.e. the horizontal axis of the output plot)
    N_t_vec  <- unique(sampling$total)
    sampling <- sampling[which(sampling$total %in% N_t_vec),]

    ###script randomly selects selected number of Sites, ###and in each site a number of samples
    dms_meta <- cbind.data.frame(sample=dms$meta$sample_names, site=dms$meta$analyses[,analysis],lat=dms$meta$lat, long=dms$meta$long)
    colnames(dms_meta) <- c("sample","site","lat","long")
    unique_Sites <- unique(na.omit(dms_meta$site)) #maximum number of Sites
    gvals <- list() #accumulate data to be saved #samples & Sites to choose, number of common alleles

    OGM_DF<-setNames(data.frame(matrix(ncol = 9, nrow = 0)), c("m","t_num_indv", "n_Sites_sel", "n_indiv_sel", "combined", "rand_Sites_sel", 
                                                             "rand_indiv_sel", "jvals", "Aprop"))
    set.seed(12345)

    for (n in 1:length(sampling$Sites)) {
    
        n_Sites_sel <- sampling$Sites[n]
        n_indiv_sel <- sampling$samples[n]
        combined <- paste0(n_Sites_sel, " ", n_indiv_sel) 
        t_num_indv <- n_Sites_sel*n_indiv_sel
     
        for (m in 1:1000) {   
            rand_Sites_sel <- unique_Sites[sample(1:length(unique_Sites))[1:n_Sites_sel]]; #print(rand_Sites_sel)  
            rand_indiv_sel <- c()
            for (s in 1:n_Sites_sel) {
                rand_indiv <- as.character(dms_meta$sample[which(dms_meta$site == rand_Sites_sel[s])][sample(1:length(which(dms_meta$site == rand_Sites_sel[s])))[1:n_indiv_sel]])
                rand_indiv_sel <- c(rand_indiv_sel,rand_indiv) ; #print(rand_indiv_sel)
            }
      
            # dms_meta$site[which(dms_meta$sample %in% rand_indiv_sel)]
            fixed_indi <- which(dms$sample_names %in% rand_indiv_sel)
            ran_vec <- rep(0, nrow(gt_sw_comp))
            ran_vec[fixed_indi] <- 1
            common_alleles  <- common_allele_count(gt_sw_comp, ran_vec)
            sw_maf    <- get_maf(gt_sw_comp)
            threshold_maf <- thresh_maf #change to higher if singleton is stronger possibility e.g Cattai
            #common allel count is 1, minor allele count observed once
            i_sw_common <- which(sw_maf > threshold_maf)
            jvals <- length( intersect( which(common_alleles[[2]] > 0), i_sw_common))
            Aprop<-jvals/length(i_sw_common)
            gvals[[m]] <- data.frame(m,t_num_indv,n_Sites_sel,n_indiv_sel,combined,rand_Sites_sel,rand_indiv_sel,jvals, Aprop) 
        }

        #save big_data2 somewhere...
        big_data2 = do.call(rbind, gvals)
        write.csv(big_data2, file = paste0(subDir,"randomisation_",n_Sites_sel,"Sites_",n_indiv_sel,"individuals.csv"), row.names = FALSE)
        OGM_DF<-rbind(OGM_DF, big_data2) 
    }
  
    clean_data <- OGM_DF %>% group_by(t_num_indv,n_Sites_sel,combined, m) %>% dplyr::summarize(Allele=mean(Aprop),Allelen=mean(jvals), samples=n()) %>% data.frame        
    clean_data$t_num_indv<-as.numeric(clean_data$t_num_indv)
    write.csv(clean_data, file = paste0(subDir, "sampling_randomisation_summary.csv"), row.names = FALSE)
    #If already run sampling randomisation use line below
    #clean_data<-read.csv(file = paste0("sampling_randomisation_summary.csv"), sep = ",")

   
   
    #claculate mininunm and maximum alleleprop for each sampling strategy

    for (e in unique(clean_data$combined)){
    min_alleledf <- min(clean_data$Allele[which(clean_data$combined==e)])
    clean_data$min_allele[which(clean_data$combined==e)] = min_alleledf
    max_alleledf <- max(clean_data$Allele[which(clean_data$combined==e)])
    clean_data$max_allele[which(clean_data$combined==e)] = max_alleledf
        }

    write.csv(clean_data, file = paste0(subDir, "sampling_randomisation_summary.csv"), row.names = FALSE)


#reorder clean_data and deteimine the first sampling combination which completely reaches over 0.9 alleleprop
clean_data <- clean_data[order(clean_data$n_Sites_sel, clean_data$t_num_indv),]
firstcombonumber <- which(clean_data$min_allele > 0.9)[1]
combointercept <- interaction(clean_data$n_Sites_sel, clean_data$t_num_indv)[firstcombonumber]


#plot a lineplot representing alleleprop vs sampling strategies
    OGM_boxplot <- ggplot(clean_data, aes(x=interaction(n_Sites_sel,t_num_indv), y=Allele)) + 
    geom_segment(data=clean_data, aes(x=interaction(n_Sites_sel,t_num_indv), y=min_allele, xend=interaction(n_Sites_sel,t_num_indv), yend=max_allele), size=5, position =  position_dodge(width = 1, preserve = "total"))+ 
    gghighlight(min(Allele) > 0.9, label_key = n_Sites_sel) +
    geom_vline(xintercept = combointercept, linetype="solid", linewidth=5, alpha=0.5, colour="darkgreen")+     
    geom_boxplot(data=clean_data,aes(x=interaction(n_Sites_sel,t_num_indv), y=min_allele), color="blue") +
    geom_boxplot(data=clean_data, aes(x=interaction(n_Sites_sel,t_num_indv), y=max_allele), color="red")+
    scale_x_discrete(guide = "axis_nested")+
     coord_cartesian(expand = FALSE)+
        scale_fill_continuous() +
        theme_bw() + scale_y_continuous(limits = c(0, 1), breaks = seq(0, 1, by = 0.1))+
        labs(title=paste0(species," sampling strategy"), subtitle = paste0("1000 Randomisations, limiting ", samplethreshold, " samples from every pop"), y="Allele Proportion", x="Number of sites \n Total samples across all sites")+
        geom_hline(yintercept = 0.9, linetype="dashed", alpha=1, colour="red")+
        #geom_segment(aes(x=combointercept, y= 0, xend=combointercept, yend=0.9), size=5, color="green")+
        theme(panel.spacing=unit(1,"lines"),
        strip.background=element_rect(color="grey30", fill="grey90"),
        panel.grid = element_blank())
ggsave(OGM_boxplot, file=paste0(subDir, species,"Sampling_Strategy_LinePlot2.png"), width = 20, height = 10, dpi = 300, units = "in", device='png')



    #read sampling combination file and select the lowest site and individual combinations for which all 100 reps are over the threshold value
    allelepropsum <- read.csv(paste0(subDir, "sampling_randomisation_summary.csv"))

    subDir <-paste0(outputloc2, "OptSiteMix/")
    if(!dir.exists(file.path(subDir))){
    
        dir.create(file.path(subDir))
    }
  
    threshlist <- allelepropsum %>% filter(allelepropsum$Allele>samplingthreshold)
    vals <- unique(threshlist$combined)
    countvals <- c()
    for (v in 1:length(vals)){  
        countvals[[v]] <- sum(threshlist$combined==vals[v]) 
        if (countvals[[v]]==100){ 
            threshlist <- threshlist 
        } else { #print("removing from df")
            threshlist <- threshlist %>% filter(threshlist$combined!=vals[v])}    
    } 

    colnames(threshlist) <- c("total_indv","num_Sites","combined","m", "allele", "num_alele", "samples")
    write.csv(threshlist, file = paste0(subDir,"List of sampling combos with every combination over ", samplingthreshold, "threshold.csv"), row.names = TRUE)

    orderbylowestsite <- threshlist[order(threshlist$num_Sites,threshlist$total_indv),]
    an <- orderbylowestsite$num_Sites[1]
    bn <- orderbylowestsite$total_indv[1]
    anbn <- cbind(an, bn)

    orderbylowestnum_indv <- threshlist[order(threshlist$total_indv, threshlist$num_Sites),]
    cn <- orderbylowestnum_indv$num_Sites[1]
    dn <- orderbylowestnum_indv$total_indv[1]
    cndn <- cbind(cn, dn)

    site_num_df <- rbind(anbn, cndn)
    colnames(site_num_df) <- c("Number of Sites", "Number of Total Samples")
    rownames(site_num_df) <- c(paste0("Minimum Site Combination Over ",samplingthreshold,"%"), paste0("Minimum Sample Combination Over ",samplingthreshold,"%"))

    write.csv(site_num_df, file = paste0(subDir,"Final Site Combinations for ", samplingthreshold, "threshold.csv"), row.names = TRUE)

    print(paste0("AlleleProp analysis for ",species," ", analysis, " complete"))



#     ██████╗ ██████╗ ████████╗███████╗██╗████████╗███████╗███╗   ███╗██╗██╗  ██╗    ███████╗██╗████████╗███████╗███████╗
#    ██╔═══██╗██╔══██╗╚══██╔══╝██╔════╝██║╚══██╔══╝██╔════╝████╗ ████║██║╚██╗██╔╝    ██╔════╝██║╚══██╔══╝██╔════╝██╔════╝
#    ██║   ██║██████╔╝   ██║   ███████╗██║   ██║   █████╗  ██╔████╔██║██║ ╚███╔╝     ███████╗██║   ██║   █████╗  ███████╗
#    ██║   ██║██╔═══╝    ██║   ╚════██║██║   ██║   ██╔══╝  ██║╚██╔╝██║██║ ██╔██╗     ╚════██║██║   ██║   ██╔══╝  ╚════██║
#    ╚██████╔╝██║        ██║   ███████║██║   ██║   ███████╗██║ ╚═╝ ██║██║██╔╝ ██╗    ███████║██║   ██║   ███████╗███████║
#     ╚═════╝ ╚═╝        ╚═╝   ╚══════╝╚═╝   ╚═╝   ╚══════╝╚═╝     ╚═╝╚═╝╚═╝  ╚═╝    ╚══════╝╚═╝   ╚═╝   ╚══════╝╚══════╝
                                                                                                                                                            
  
  
    #Now check from the paratmeters file to see if the user wants minimise the number of Sites, and/or minimise the total number of individuals 
    #and run the OptSiteMix for the reauested parameters

  
    if ((OptSiteMixMinSites == 'yes') == TRUE |(OptSiteMixMinSites == 'Yes') == TRUE |(OptSiteMixMinSites == 'Y') == TRUE){ 
        print("Running OptSiteMix for minimum Sites")
        subDir <-paste0(outputloc2, "OptSiteMix/Minimum_Sites/")
        if(!dir.exists(file.path(subDir))){
            dir.create(file.path(subDir))
        }
    
        gta  <- dms$gt
        pops <- as.vector(na.omit(dms$meta$analyses[,analysis]))    
        tmpmac <- gt_to_pop_minor_allele_counts(gta, pops)  

        if (any(which(colSums(tmpmac$N < 2)>0)) == TRUE | any(which(colSums(tmpmac$MAC == 0)==nrow(tmpmac$MAC))) == TRUE) {
            i_loc_miss <- which(colSums(tmpmac$N < 2)>0)
            i_mac_zero <- which(colSums(tmpmac$MAC == 0)==nrow(tmpmac$MAC))
            i_rm <- union(i_loc_miss, i_mac_zero)
            gta <- gta[, -i_rm]  
            pmac <- gt_to_pop_minor_allele_counts(gta, pops)
        } else {
            pmac <- gt_to_pop_minor_allele_counts(gta, pops)
        }
    
    
        uq <- length(unique(pops)) 
        opt_psfs_MAC <- list()               
        minw <- rep(0, uq)

        for ( m in 1:length(an) ) {
            N_t <- as.numeric(an[m])
            cat("\n Running ", N_t, " ...\n") 
            max_wts <- rep(1, uq)
            f <- uq-N_t
            initial_wts <- sample(c(rep(1,N_t),rep(0, f)))
            #N_t_MACvar <- paste0("opt_psfs_MAC_", N_t)
            opt_psfs_MAC[[m]] <- optimize_single_objective( gt=pmac$MAC, N_t=N_t, initial_weights=initial_wts, weights_max=max_wts, measure="psfs", max_steps=num_steps, max_t=0.005, m=20, p_depends_delta=TRUE, pMAC_mode=TRUE, Nmat=pmac$N, weights_min=minw)
            png(file = paste0(subDir, species, analysis,"optpsfs_minSites_MAC_temperature_", N_t, "_Sites.png"), width = 13.3, height = 7.5, res = 600, units = "in") 
            plot(opt_psfs_MAC[[m]]$value)         
            dev.off()    

            if (length(an) >1){   
                ### Generate table of optimised site selection
                solution_table_site <- mat.or.vec(nrow(pmac$MAC), length(an))
                for (i in 1:length(an)) {
                    solution_table_site[,i] <- cbind(opt_psfs_MAC[[i]]$weight[num_steps,])            
                }
            }else{
                solution_table_site = NULL
                for (i in 1:length(an)) {
                    solution_table_site <- as.data.frame(cbind(opt_psfs_MAC[[i]]$weight[num_steps,]))
                }
            }   
      
            colnames(solution_table_site) <- paste0("Sites")
            rownames(solution_table_site) <- rownames(pmac$MAC)     
            write.table(solution_table_site, paste0(subDir, species, analysis," ogmsfs min Sites.csv"),sep=",",quote=FALSE, row.name=TRUE,col.name=TRUE)
            
            #plot optimised site selection
            Opt_solution_pdf <- c() 
            for (ib in 1:length(an)) {
                Opt_solution_pdf[ib] <- opt_psfs_MAC[[ib]]$value[num_steps]
            }
      
      
            Opt_solution_pdffinal <- as.data.frame(cbind(paste0("Sites"), Opt_solution_pdf))
            colnames(Opt_solution_pdffinal) <- c("Sites", "sfs_prop_m")
            Opt_Solution_Final <- as.data.frame(lapply(Opt_solution_pdffinal, unlist))
            Opt_Solution_Final$sfs_prop_m <- as.numeric(Opt_Solution_Final$sfs_prop_m)
            Opt_Solution_Final$Sites <- fct_inorder(Opt_Solution_Final$Sites)
            opt_solution_dotplot <- ggplot(Opt_Solution_Final, aes(x=Sites, y=sfs_prop_m)) + geom_dotplot(binaxis='y', binwidth=0.005, stackdir='center') + labs(y="SFS Proportion m", x="Sites")
            opt_solution_dotplot
            ### Randomisation script - generates randomised site selection to compare to optimised site selection
            rand_results <- c()
            solution_pdf <- c()
            rand_results_val <- c()

            for ( ic in 1:length(an) ) {   
        
                iNt <- as.numeric(an[ic])    
        
                for (j in 1:rand_numsteps) {      
                    f <- uq-iNt       
                    initial_wts <- sample(c(rep(1,iNt),rep(0, f)))   
                    RandomSites <- sample(1:uq)[1:iNt]
                    rand_minw <- rep(0, uq)
                    rand_minw[RandomSites] = 1
                    rand_results[[j]] <- optimize_single_objective( gt=pmac$MAC, N_t=iNt, initial_weights=initial_wts, weights_max=max_wts, measure="psfs", max_steps=1, max_t=0.005, m=20, p_depends_delta=TRUE, pMAC_mode=TRUE, Nmat=pmac$N, weights_min=rand_minw)
                    rand_results_val[j] <- as.data.frame(rand_results[[j]]$value)
                }
                rand_results_valfinal <- cbind((paste0("Sites")), rand_results_val)
                solution_pdf <- as.data.frame(rbind(solution_pdf, rand_results_valfinal))
            }
      
            colnames(solution_pdf) <- c("Sites", "sfs_prop_m")
            solution_pdf2 <- as.data.frame(lapply(solution_pdf, unlist))
            solution_pdf2$Sites <- fct_inorder(solution_pdf2$Sites)
            rand_solution_plot <- ggplot(solution_pdf2, aes(x=Sites, y=sfs_prop_m))+ geom_violin() + labs(y="SFS Proportion m", x="Sites")
            write.table(solution_pdf2, paste0(subDir, species, analysis," min Sites Randomisation Table.csv"),sep=",",quote=FALSE, row.name=FALSE,col.name=TRUE)
            write.table(Opt_Solution_Final, paste0(subDir, species, analysis," Optimised min Sites Table.csv"),sep=",",quote=FALSE, row.name=FALSE,col.name=TRUE)

            ### Combined plot of optimised vs randomised site selection
            combined_plot <- ggplot(solution_pdf2, aes(x=Sites, y=sfs_prop_m)) + 
            geom_violin() + 
            geom_dotplot(data=Opt_Solution_Final, binaxis='y', binwidth=0.005, stackdir='center', fill="red") + 
            labs(title=paste0("Optimised vs Random for ", iNt, "Sites"), subtitle= paste0(species," ", analysis), y="SFS Proportion (m)", x="Sites Selected (n)") + 
            theme_minimal(base_size=20) + 
            theme(axis.title.y = element_text(margin = margin(t = 0, r = 20, b = 0, l = 0)), axis.title.x = element_text(margin = margin(t = 20, r = 0, b = 0, l = 0)),  plot.title = element_text(margin = margin(t = 0, r = 0, b = 10, l = 0)), plot.subtitle = element_text(margin = margin(t = 0, r = 0, b = 20, l = 0)))    
            combined_plot
            ggsave(paste0(species, analysis," optimised min Sites VS random.pdf"), path=paste0(subDir), width = 13, height = 12, dpi = 600, units = "in")

            #Now take the optimised site combination and input these as forced allele proportion Sites 
            #to compare between optimised vs random allele proportion      
            solution_table_site$row_names <- row.names(solution_table_site)
            OrderforOptimisedSites <- solution_table_site[order(solution_table_site$Sites, decreasing =TRUE),]
            OptimisedSites <- OrderforOptimisedSites$row_names[1:an]

            ###if you want to remove all missingness (recommended sometimes use the first hashed out section. otherwise use the second)
            #cones GT matrix in new variable (ColID are SNPIDs)               
            #create a temp meta dataframe to remove Sites/samples that include less than X individuals
            temp<-as.data.frame(m1$analyses)
            # Below lines remove Sites with less than threshold number of samples, and subsample Sites that are above the threshold  
            samplethreshold <-5 # chnage this to what number of samples you want toi have uniform across all pops  
            for (x in unique(na.omit(temp[,analysis]))) {     
                pop_samples <- which(temp[,analysis] == x)    
                if (length(pop_samples) < samplethreshold) {     
                    temp[, analysis]<-gsub(x, replacement=NA, temp[, analysis])      
                    #print(paste0("Removing ",x, " with ", " with a sample size of ",length(pop_samples)," samples"))    
                }    
                if (length(pop_samples) == samplethreshold) {      
                    #print(paste0("Keeping ",x, " with ", " with a sample size of ",length(pop_samples)," samples"))    
                }    
                if (length(pop_samples) > samplethreshold) {      
                    subsamplepops <- pop_samples[sample(1:length(pop_samples), size=samplethreshold, replace = F)]      
                    for (b in 1:length(pop_samples)){        
                        if (pop_samples[b] %in% subsamplepops == FALSE){          
                            temp[, analysis][pop_samples[b]] <- NA          
                        }        
                        #print(paste0("Reducing ",x, " to a sample size of ", samplethreshold," samples"))      
                    }    
                } else {}  
            }               
        
            m1$analyses<-temp      
            dm <- dart.meta.data.merge(d3, m1)      
            fields    <- c(analysis)      
            dms     <- data.by.meta.fields(dm, fields, RandRbase, species, dataset, object=analysis); #print(dms$meta$analyses[,analysis])     
            #correct inconsistencies in J's scripts for naming treatment which is for naming files
            treatment <- paste0("raw_SNPFilt_1SNPperClone_Field_", analysis)         
            dms$treatment <- treatment         
            gta <- dms$gt             
            gt_sw_comp <- gta    
            ncol(gt_sw_comp)    

            ###load required code
            source_url("https://github.com/RichardDimon/get_maf/blob/main/get_maf.r?raw=TRUE")
            source_url("https://github.com/RichardDimon/common_allele_count/blob/main/common_allele_count.r?raw=TRUE")
            ###the next line of code runs a loop that create a table with all 
            ###possible combinations of sampling assuming all included pops have atleast 5 samples available
            sampling<-setNames(data.frame(matrix(ncol = 3, nrow = 0)), c("Sites", "samples", "total"))
            for (c in 1:length(unique(dms$meta$analyses[,analysis]))) { 
                for (ind in 1:samplethreshold) {
                    sampling[nrow(sampling) + 1,] = list(c,ind,c*ind) 
                }
            }
      
            ###this bit orders the sampling strategy and removes strategies with less than x samples
            sampling<-sampling[order(sampling$total),]
            ###script randomly selects selected number of Sites and in each site a number of samples
            dms_meta <- cbind.data.frame(sample=dms$meta$sample_names, site=dms$meta$analyses[,analysis],lat=dms$meta$lat, long=dms$meta$long)
            colnames(dms_meta) <- c("sample","site","lat","long")
            unique_Sites <- unique(na.omit(dms_meta$site)) #maximum number of Sites
            gvals <- list() #accumulate data to be saved #samples & Sites to choose, number of common alleles

            #insert loop here to run through table listing parameters in bottom two lines#
            OGM_DF<-setNames(data.frame(matrix(ncol = 8, nrow = 0)), c("m","t_num_indv", "n_Sites_sel", "n_indiv_sel", "rand_Sites_sel", 
                                                             "rand_indiv_sel", "jvals", "Aprop"))
            set.seed(12345)   
            ###Set Sites to select by name and set numbers of Sites and individuals to select###
            Sites_sel <- OptimisedSites
            #Sites_sel <- unique(na.omit(dms_meta$site))       
            rand_indiv_sel <- c()
            n_Sites_sel <- as.numeric(an)
            t_num_indv <- as.numeric(bn)
            n_indiv_sel <- t_num_indv/n_Sites_sel         

            for (m in 1:1000) {
                rand_Sites_sel <- Sites_sel[sample(1:length(Sites_sel))[1:n_Sites_sel]]; #print(rand_Sites_sel)  
                rand_indiv_sel <- c()      
                for (s in 1:n_Sites_sel) {    
                    rand_indiv <- as.character(dms_meta$sample[which(dms_meta$site == rand_Sites_sel[s])][sample(1:length(which(dms_meta$site == rand_Sites_sel[s])))[1:n_indiv_sel]])     
                    rand_indiv_sel <- c(rand_indiv_sel,rand_indiv) ; #print(rand_indiv_sel)  
                }    
        
       
                # dms_meta$site[which(dms_meta$sample %in% rand_indiv_sel)]
                fixed_indi <- which(dms$sample_names %in% rand_indiv_sel)      
                ran_vec <- rep(0, nrow(gt_sw_comp))  
                ran_vec[fixed_indi] <- 1 
                common_alleles  <- common_allele_count(gt_sw_comp, ran_vec)     
                sw_maf    <- get_maf(gt_sw_comp) 
                threshold_maf <- thresh_maf #change to higher if singleton is stronger possibility e.g Cattai   
                #common allel count is 1, minor allele count observed once  
                i_sw_common <- which(sw_maf > threshold_maf)  
                jvals <- length( intersect( which(common_alleles[[2]] > 0), i_sw_common))   
                Aprop<-jvals/length(i_sw_common)       
                gvals[[m]] <- data.frame(m,t_num_indv,n_Sites_sel,n_indiv_sel,rand_Sites_sel,rand_indiv_sel,jvals, Aprop) 
            }
      
      
            #save big_data2 somewhere... 
            big_data2 = do.call(rbind, gvals)
            write.csv(big_data2, file = paste0(subDir,"forced",n_Sites_sel,"Sites_",n_indiv_sel,"individuals.csv"), row.names = FALSE) 
            OGM_DF<-rbind(OGM_DF, big_data2)
            clean_data_optimised <- OGM_DF %>% group_by(t_num_indv,n_Sites_sel,m) %>% summarize(Average_Allele=mean(Aprop),Average_Allelen=mean(jvals)) %>% data.frame
            #clean_data$t_num_indv<-as.numeric(clean_data$t_num_indv)
            write.csv(clean_data_optimised, file = paste0(subDir, "Optimised_AlleleProp_Summary.csv"), row.names = FALSE)

            #Now repreat the same using randomisation of all Sites within the dataset (not just the selected forced Sites)
            Sites_sel <- unique(na.omit(dms_meta$site))      
            rand_indiv_sel <- c()

            for (m in 1:1000) {
                rand_Sites_sel <- Sites_sel[sample(1:length(Sites_sel))[1:n_Sites_sel]]; #print(rand_Sites_sel)
                rand_indiv_sel <- c()          
                for (s in 1:n_Sites_sel) {    
                    rand_indiv <- as.character(dms_meta$sample[which(dms_meta$site == rand_Sites_sel[s])][sample(1:length(which(dms_meta$site == rand_Sites_sel[s])))[1:n_indiv_sel]])   
                    rand_indiv_sel <- c(rand_indiv_sel,rand_indiv) ; #print(rand_indiv_sel)  
                }    

                # dms_meta$site[which(dms_meta$sample %in% rand_indiv_sel)]   
                fixed_indi <- which(dms$sample_names %in% rand_indiv_sel)     
                ran_vec <- rep(0, nrow(gt_sw_comp))   
                ran_vec[fixed_indi] <- 1   
                common_alleles  <- common_allele_count(gt_sw_comp, ran_vec)       
                sw_maf    <- get_maf(gt_sw_comp)   
                threshold_maf <- thresh_maf #change to higher if singleton is stronger possibility e.g Cattai  
                #common allel count is 1, minor allele count observed once  
                i_sw_common <- which(sw_maf > threshold_maf)  
                jvals <- length( intersect( which(common_alleles[[2]] > 0), i_sw_common))  
                Aprop<-jvals/length(i_sw_common)       
                gvals[[m]] <- data.frame(m,t_num_indv,n_Sites_sel,n_indiv_sel,rand_Sites_sel,rand_indiv_sel,jvals, Aprop) 
            }  
      
      
            #save big_data2 somewhere...
            big_data2 = do.call(rbind, gvals)
            write.csv(big_data2, file = paste0(subDir,"randomisation_",n_Sites_sel,"Sites_",n_indiv_sel,"individuals.csv"), row.names = FALSE)
            OGM_DF<-rbind(OGM_DF, big_data2)
            clean_data_random <- OGM_DF %>% group_by(t_num_indv,n_Sites_sel,m) %>% summarize(Average_Allele=mean(Aprop),Average_Allelen=mean(jvals)) %>% data.frame
            #clean_data$t_num_indv<-as.numeric(clean_data$t_num_indv)
            write.csv(clean_data_random, file = paste0(subDir, "Random_AlleleProp_Summary.csv"), row.names = FALSE)


            
            
        }
            
        #FORCED SITE Optimisation
        if ((ForcedOptSiteMix == 'yes') == TRUE |(ForcedOptSiteMix == 'Yes') == TRUE |(ForcedOptSiteMix == 'Y') == TRUE ){
            #reads the string variable from the parameters file and forced optimisation selection in the binary format for minw
            ForcedOptSites <- str_split(ListForcedSites, ",", simplify=TRUE);
            uqpops <- unique(pops)
            
            #force site number and total samples if required below:
            #n <- 3
            #n <-5
            
            if (length(ForcedOptSites) > an) {
                print(paste0("Warning: too many forced sites! need to supply ", an, " or fewer sites"))
            } else { 
                for (p in 1:length(ForcedOptSites)){
                    uqpops <- gsub(ForcedOptSites[p], "1", uqpops)
                }
                for (v in 1:length(uqpops)){
                    if (uqpops[v] != 1){
                        uqpops <- gsub(uqpops[v], "0", uqpops)
                    } else {}    
                }
                minw <- uqpops
                print("Running Forced OptSiteMix for minimum sites")        
                subDir <-paste0(outputloc2, "OptSiteMix/Minimum_Sites/ForcedSiteSelection/")
                if(!dir.exists(file.path(subDir))){
                    dir.create(file.path(subDir))
                }


                gta  <- dms$gt
                pops <- as.vector(na.omit(dms$meta$analyses[,analysis]))    
                tmpmac <- gt_to_pop_minor_allele_counts(gta, pops)    

                if (any(which(colSums(tmpmac$N < 2)>0)) == TRUE | any(which(colSums(tmpmac$MAC == 0)==nrow(tmpmac$MAC))) == TRUE) {
                    i_loc_miss <- which(colSums(tmpmac$N < 2)>0)
                    i_mac_zero <- which(colSums(tmpmac$MAC == 0)==nrow(tmpmac$MAC))
                    i_rm <- union(i_loc_miss, i_mac_zero)
                    gta <- gta[, -i_rm]  
                    pmac <- gt_to_pop_minor_allele_counts(gta, pops)
                } else {
                    pmac <- gt_to_pop_minor_allele_counts(gta, pops)
                }

                uq <- length(unique(pops)) 
                opt_psfs_MAC <- list()
                print("running forced site selection...")

            }

            for ( m in 1:length(an) ) {

                N_t <- as.numeric(an[m])
                cat("\n Running ", N_t, " ...\n")
                max_wts <- rep(1, uq)
                f <- uq-N_t
                initial_wts <- sample(c(rep(1,N_t),rep(0, f)))
                #N_t_MACvar <- paste0("opt_psfs_MAC_", N_t)
                opt_psfs_MAC[[m]] <- optimize_single_objective( gt=pmac$MAC, N_t=N_t, initial_weights=initial_wts, weights_max=max_wts, measure="psfs", max_steps=num_steps, max_t=0.005, m=20, p_depends_delta=TRUE, pMAC_mode=TRUE, Nmat=pmac$N, weights_min=minw)       

                png(file = paste0(subDir, species, analysis,"optpsfs_minsites_MAC_temperature_", N_t, "_Forced_Sites.png"), width = 13.3, height = 7.5, res = 600, units = "in")
                plot(opt_psfs_MAC[[m]]$value)         
                dev.off()

                if (length(an) >1){   
                    ### Generate table of optimised site selection
                    solution_table_samp <- mat.or.vec(nrow(pmac$MAC), length(an))
                    for (i in 1:length(an)) {
                        solution_table_samp[,i] <- cbind(opt_psfs_MAC[[i]]$weight[num_steps,])        
                    }
                }else{
                    solution_table_samp = NULL
                    for (i in 1:length(an)) {   
                        solution_table_samp <- as.data.frame(cbind(opt_psfs_MAC[[i]]$weight[num_steps,]))       
                    }
                }   

                colnames(solution_table_samp) <- paste0("Sites")
                rownames(solution_table_samp) <- rownames(pmac$MAC)     
                write.table(solution_table_samp, paste0(subDir, species, analysis," ogmsfs min sites"),sep=",",quote=FALSE, row.name=TRUE,col.name=NA)

                #plot optimised site selection
                Opt_solution_pdf <- c() 
                for (ib in 1:length(an)) {

                    Opt_solution_pdf[ib] <- opt_psfs_MAC[[ib]]$value[num_steps]
                }


                Opt_solution_pdffinal <- as.data.frame(cbind(paste0("Sites"), Opt_solution_pdf))
                colnames(Opt_solution_pdffinal) <- c("Sites", "sfs_prop_m")
                Opt_Solution_Final <- as.data.frame(lapply(Opt_solution_pdffinal, unlist))
                Opt_Solution_Final$sfs_prop_m <- as.numeric(Opt_Solution_Final$sfs_prop_m)
                Opt_Solution_Final$Sites <- fct_inorder(Opt_Solution_Final$Sites)
                opt_solution_dotplot <- ggplot(Opt_Solution_Final, aes(x=Sites, y=sfs_prop_m)) + geom_dotplot(binaxis='y', binwidth=0.005, stackdir='center') + labs(y="SFS Proportion m", x="Sites")
                opt_solution_dotplot

                ### Randomisation script - generates randomised site selection to compare to optimised site selection
                rand_results <- c()
                solution_pdf <- c()
                rand_results_val <- c()

                for ( ic in 1:length(an) ) {

                    iNt <- as.numeric(an[ic])
                    for (j in 1:rand_numsteps) {
                        f <- uq-iNt
                        initial_wts <- sample(c(rep(1,iNt),rep(0, f)))
                        RandomSites <- sample(1:uq)[1:iNt]
                        rand_minw <- rep(0, uq)
                        rand_minw[RandomSites] = 1
                        rand_results[[j]] <- optimize_single_objective( gt=pmac$MAC, N_t=iNt, initial_weights=initial_wts, weights_max=max_wts, measure="psfs", max_steps=1, max_t=0.005, m=20, p_depends_delta=TRUE, pMAC_mode=TRUE, Nmat=pmac$N, weights_min=rand_minw)
                        rand_results_val[j] <- as.data.frame(rand_results[[j]]$value)    
                    }
                    rand_results_valfinal <- cbind((paste0("Sites")), rand_results_val)
                    solution_pdf <- as.data.frame(rbind(solution_pdf, rand_results_valfinal))
                }


                colnames(solution_pdf) <- c("Sites", "sfs_prop_m")
                solution_pdf2 <- as.data.frame(lapply(solution_pdf, unlist))
                solution_pdf2$Sites <- fct_inorder(solution_pdf2$Sites)
                rand_solution_plot <- ggplot(solution_pdf2, aes(x=Sites, y=sfs_prop_m))+ geom_violin() + labs(y="SFS Proportion m", x="Sites")

                write.table(solution_pdf2, paste0(subDir, species, analysis," min sites Randomisation Table.csv"),sep=",",quote=FALSE, row.name=FALSE,col.name=TRUE)
                write.table(Opt_Solution_Final, paste0(subDir, species, analysis," Forced Optimised min sites Table.csv"),sep=",",quote=FALSE, row.name=FALSE,col.name=TRUE)

                ### Combined plot of optimised vs randomised site selection
                combined_plot <- ggplot(solution_pdf2, aes(x=Sites, y=sfs_prop_m)) + 
                geom_violin() + 
                geom_dotplot(data=Opt_Solution_Final, binaxis='y', binwidth=0.005, stackdir='center', fill="red") + 
                labs(title=paste0("Forced Optimised vs Random for ", iNt, " Sites"), subtitle= paste0(species," ", analysis), y="SFS Proportion (m)", x="Sites Selected (n)") + 
                theme_minimal(base_size=20) + 
                theme(axis.title.y = element_text(margin = margin(t = 0, r = 20, b = 0, l = 0)), axis.title.x = element_text(margin = margin(t = 20, r = 0, b = 0, l = 0)),  plot.title = element_text(margin = margin(t = 0, r = 0, b = 10, l = 0)), plot.subtitle = element_text(margin = margin(t = 0, r = 0, b = 20, l = 0)))    
                combined_plot
                ggsave(paste0(species, analysis," forced optimised min sites VS random.pdf"), path=paste0(subDir), width = 13, height = 12, dpi = 600, units = "in")  


                #Now take the optimised site combination and input these as forced allele proportion Sites 
                #to compare between optimised vs random allele proportion      
                solution_table_samp$row_names <- row.names(solution_table_samp)
                OrderforforcedOptimisedSites <- solution_table_samp[order(solution_table_samp$Sites, decreasing =TRUE),]
                forcedOptimisedSites <- OrderforforcedOptimisedSites$row_names[1:an]

                #create a temp meta dataframe to remove Sites/samples that include less than X individuals
                temp<-as.data.frame(m1$analyses)

                # Below lines remove Sites with less than threshold number of samples, and subsample Sites that are above the threshold  
                samplethreshold <-5 # chnage this to what number of samples you want toi have uniform across all pops  
                for (x in unique(na.omit(temp[,analysis]))) {    
                    pop_samples <- which(temp[,analysis] == x)    
                    if (length(pop_samples) < samplethreshold) {     
                        temp[, analysis]<-gsub(x, replacement=NA, temp[, analysis])      
                        #print(paste0("Removing ",x, " with ", " with a sample size of ",length(pop_samples)," samples"))    
                    }    
                    if (length(pop_samples) == samplethreshold) {      

                        #print(paste0("Keeping ",x, " with ", " with a sample size of ",length(pop_samples)," samples"))    
                    }    
                    if (length(pop_samples) > samplethreshold) {      
                        subsamplepops <- pop_samples[sample(1:length(pop_samples), size=samplethreshold, replace = F)]      
                        for (b in 1:length(pop_samples)){        
                            if (pop_samples[b] %in% subsamplepops == FALSE){         
                                temp[, analysis][pop_samples[b]] <- NA        
                            }        
                            #print(paste0("Reducing ",x, " to a sample size of ", samplethreshold," samples"))      
                        }    
                    } else {}    
                }               

                m1$analyses<-temp      
                dm <- dart.meta.data.merge(d3, m1)      
                fields    <- c(analysis)      
                dms     <- data.by.meta.fields(dm, fields, RandRbase, species, dataset, object=analysis); #print(dms$meta$analyses[,analysis])     
                #correct inconsistencies in J's scripts for naming treatment which is for naming files
                treatment <- paste0("raw_SNPFilt_1SNPperClone_Field_", analysis)         
                dms$treatment <- treatment         
                gta <- dms$gt             
                gt_sw_comp <- gta    
                ncol(gt_sw_comp) 

                ###load required code
                source_url("https://github.com/RichardDimon/get_maf/blob/main/get_maf.r?raw=TRUE")
                source_url("https://github.com/RichardDimon/common_allele_count/blob/main/common_allele_count.r?raw=TRUE")
                ###the next line of code runs a loop that create a table with all 
                ###possible combinations of sampling assuming all included pops have atleast 5 samples available
                sampling<-setNames(data.frame(matrix(ncol = 3, nrow = 0)), c("Sites", "samples", "total"))
                for (c in 1:length(unique(dms$meta$analyses[,analysis]))) { 
                    for (ind in 1:samplethreshold) {
                        sampling[nrow(sampling) + 1,] = list(c,ind,c*ind) 
                    }
                }
                ###this bit orders the sampling strategy and removes strategies with less than x samples
                sampling<-sampling[order(sampling$total),]
                ###script randomly selects selected number of Sites and in each site a number of samples
                dms_meta <- cbind.data.frame(sample=dms$meta$sample_names, site=dms$meta$analyses[,analysis],lat=dms$meta$lat, long=dms$meta$long)
                colnames(dms_meta) <- c("sample","site","lat","long")
                unique_Sites <- unique(na.omit(dms_meta$site)) #maximum number of Sites
                gvals <- list() #accumulate data to be saved #samples & Sites to choose, number of common alleles

                #insert loop here to run through table listing parameters in bottom two lines#
                OGM_DF<-setNames(data.frame(matrix(ncol = 8, nrow = 0)), c("m","t_num_indv", "n_Sites_sel", "n_indiv_sel", "rand_Sites_sel", 
                "rand_indiv_sel", "jvals", "Aprop"))
                set.seed(12345)   
                ###Set Sites to select by name and set numbers of Sites and individuals to select###
                Sites_sel <- forcedOptimisedSites
                #Sites_sel <- unique(na.omit(dms_meta$site))       
                rand_indiv_sel <- c()
                n_Sites_sel <- as.numeric(an)
                t_num_indv <- as.numeric(bn)
                n_indiv_sel <- t_num_indv/n_Sites_sel         

                for (m in 1:1000) {
                    rand_Sites_sel <- Sites_sel[sample(1:length(Sites_sel))[1:n_Sites_sel]]; #print(rand_Sites_sel)  
                    rand_indiv_sel <- c()      

                    for (s in 1:n_Sites_sel) {    
                        rand_indiv <- as.character(dms_meta$sample[which(dms_meta$site == rand_Sites_sel[s])][sample(1:length(which(dms_meta$site == rand_Sites_sel[s])))[1:n_indiv_sel]])     
                        rand_indiv_sel <- c(rand_indiv_sel,rand_indiv) ; #print(rand_indiv_sel)  
                    }    

                        # dms_meta$site[which(dms_meta$sample %in% rand_indiv_sel)]
                        fixed_indi <- which(dms$sample_names %in% rand_indiv_sel)      
                        ran_vec <- rep(0, nrow(gt_sw_comp))  
                        ran_vec[fixed_indi] <- 1 
                        common_alleles  <- common_allele_count(gt_sw_comp, ran_vec)     
                        sw_maf    <- get_maf(gt_sw_comp) 
                        threshold_maf <- thresh_maf #change to higher if singleton is stronger possibility e.g Cattai   
                        #common allel count is 1, minor allele count observed once  
                        i_sw_common <- which(sw_maf > threshold_maf)  
                        jvals <- length( intersect( which(common_alleles[[2]] > 0), i_sw_common))   
                        Aprop<-jvals/length(i_sw_common)       
                        gvals[[m]] <- data.frame(m,t_num_indv,n_Sites_sel,n_indiv_sel,rand_Sites_sel,rand_indiv_sel,jvals, Aprop) 
                }


                #save big_data2 somewhere... 
                big_data2 = do.call(rbind, gvals)
                write.csv(big_data2, file = paste0(subDir,"forced",n_Sites_sel,"Sites_",n_indiv_sel,"individuals.csv"), row.names = FALSE) 
                OGM_DF<-rbind(OGM_DF, big_data2)
                clean_data_forcedoptimised <- OGM_DF %>% group_by(t_num_indv,n_Sites_sel,m) %>% summarize(Average_Allele=mean(Aprop),Average_Allelen=mean(jvals)) %>% data.frame
                #clean_data$t_num_indv<-as.numeric(clean_data$t_num_indv)
                write.csv(clean_data_forcedoptimised, file = paste0(subDir, "Forced_Optimisation_AlleleProp_Summary.csv"), row.names = FALSE)



                #now combine all results into a summmary csv file (with forced optimisation)                                         
                final_output <- cbind.data.frame(analysis, "Minimum Sites", samplingthreshold, an, bn/an, bn, paste0(OptimisedSites, collapse=', '), clean_data_optimised$Average_Allele, clean_data_optimised$Average_Allelen, ListForcedSites, paste0(forcedOptimisedSites, collapse=', '), clean_data_forcedoptimised$Average_Allele, clean_data_forcedoptimised$Average_Allelen, clean_data_random$Average_Allele, clean_data_random$Average_Allelen)     
                colnames(final_output) <- c("Analysis", "Selection", "Sample Threshold", "N Sites",  "N Samples per Site", "N Total Samples", "Optimised Sites", "Optimised Allele Prop", "Optimised N Alleles", "Intial Forced Sites", "Forced Optimised Sites", "Forced Optimised Allele Prop", "Forced Optimised N Alleles", "Random Allele Prop", "Random N Alleles")        
                subDir <-paste0(outputloc2, "OptSiteMix/")
                write.csv(final_output, file = paste0(subDir, "Forced_OptSiteMix_MinSites_Final_Output.csv"), row.names= FALSE)   

            }

        } else {
            #now combine all results into a summmary csv file                                          
            final_output <- cbind.data.frame(analysis, "Minimum Sites", samplingthreshold, an, bn/an, bn, paste0(OptimisedSites, collapse=', '), clean_data_optimised$Average_Allele, clean_data_optimised$Average_Allelen, clean_data_random$Average_Allele, clean_data_random$Average_Allelen)     
            colnames(final_output) <- c("Analysis", "Selection", "Sample Threshold", "N Sites",  "N Samples per Site", "N Total Samples", "Optimised Sites", "Optimised Allele Prop", "Optimised N Alleles", "Random Allele Prop", "Random N Alleles")        
            subDir <-paste0(outputloc2, "OptSiteMix/")
            write.csv(final_output, file = paste0(subDir, "OptSiteMix_MinSites_Final_Output.csv"), row.names= FALSE)    
        }   
    } else {
        print("not running OptSiteMix for minimum Sites")
    }
  


#     ██████╗ ██████╗ ████████╗███████╗██╗████████╗███████╗███╗   ███╗██╗██╗  ██╗    ███████╗ █████╗ ███╗   ███╗██████╗ ██╗     ███████╗███████╗
#    ██╔═══██╗██╔══██╗╚══██╔══╝██╔════╝██║╚══██╔══╝██╔════╝████╗ ████║██║╚██╗██╔╝    ██╔════╝██╔══██╗████╗ ████║██╔══██╗██║     ██╔════╝██╔════╝
#    ██║   ██║██████╔╝   ██║   ███████╗██║   ██║   █████╗  ██╔████╔██║██║ ╚███╔╝     ███████╗███████║██╔████╔██║██████╔╝██║     █████╗  ███████╗
#    ██║   ██║██╔═══╝    ██║   ╚════██║██║   ██║   ██╔══╝  ██║╚██╔╝██║██║ ██╔██╗     ╚════██║██╔══██║██║╚██╔╝██║██╔═══╝ ██║     ██╔══╝  ╚════██║
#    ╚██████╔╝██║        ██║   ███████║██║   ██║   ███████╗██║ ╚═╝ ██║██║██╔╝ ██╗    ███████║██║  ██║██║ ╚═╝ ██║██║     ███████╗███████╗███████║
#     ╚═════╝ ╚═╝        ╚═╝   ╚══════╝╚═╝   ╚═╝   ╚══════╝╚═╝     ╚═╝╚═╝╚═╝  ╚═╝    ╚══════╝╚═╝  ╚═╝╚═╝     ╚═╝╚═╝     ╚══════╝╚══════╝╚══════╝
                                                                                                                                           
  
  
    if ((OptSiteMixMinSamples == 'yes') == TRUE |(OptSiteMixMinSamples == 'Yes') == TRUE |(OptSiteMixMinSamples == 'Y') == TRUE ){
        print("Running OptSiteMix for minimum samples")    
        subDir <-paste0(outputloc2, "OptSiteMix/Minimum_Samples/")
        if(!dir.exists(file.path(subDir))){
            dir.create(file.path(subDir))
        }

        gta  <- dms$gt
        pops <- as.vector(na.omit(dms$meta$analyses[,analysis]))    
        tmpmac <- gt_to_pop_minor_allele_counts(gta, pops)    

        if (any(which(colSums(tmpmac$N < 2)>0)) == TRUE | any(which(colSums(tmpmac$MAC == 0)==nrow(tmpmac$MAC))) == TRUE) {
            i_loc_miss <- which(colSums(tmpmac$N < 2)>0)
            i_mac_zero <- which(colSums(tmpmac$MAC == 0)==nrow(tmpmac$MAC))
            i_rm <- union(i_loc_miss, i_mac_zero)
            gta <- gta[, -i_rm]  
            pmac <- gt_to_pop_minor_allele_counts(gta, pops)
        } else {
            pmac <- gt_to_pop_minor_allele_counts(gta, pops)
        }
        uq <- length(unique(pops)) 
        opt_psfs_MAC <- list()
        minw <- rep(0, uq)        
    
    
        for ( m in 1:length(cn) ) {
            N_t <- as.numeric(cn[m])
            cat("\n Running ", N_t, " ...\n")
            max_wts <- rep(1, uq)
            f <- uq-N_t
            initial_wts <- sample(c(rep(1,N_t),rep(0, f)))
            #N_t_MACvar <- paste0("opt_psfs_MAC_", N_t)
            opt_psfs_MAC[[m]] <- optimize_single_objective( gt=pmac$MAC, N_t=N_t, initial_weights=initial_wts, weights_max=max_wts, measure="psfs", max_steps=num_steps, max_t=0.005, m=20, p_depends_delta=TRUE, pMAC_mode=TRUE, Nmat=pmac$N, weights_min=minw)       
            png(file = paste0(subDir, species, analysis,"optpsfs_minsamples_MAC_temperature_", N_t, "_Sites.png"), width = 13.3, height = 7.5, res = 600, units = "in")
            plot(opt_psfs_MAC[[m]]$value)         
            dev.off()     

            if (length(cn) >1){   
                ### Generate table of optimised site selection
                solution_table_samp <- mat.or.vec(nrow(pmac$MAC), length(cn))
                for (i in 1:length(cn)) {
                    solution_table_samp[,i] <- cbind(opt_psfs_MAC[[i]]$weight[num_steps,])        
                }
            }else{
                solution_table_samp = NULL
                for (i in 1:length(cn)) {   
                    solution_table_samp <- as.data.frame(cbind(opt_psfs_MAC[[i]]$weight[num_steps,]))       
                }
            }   
      
            colnames(solution_table_samp) <- paste0("Sites")
            rownames(solution_table_samp) <- rownames(pmac$MAC)     
            write.table(solution_table_samp, paste0(subDir, species, analysis," ogmsfs min samples.csv"),sep=",",quote=FALSE, row.name=TRUE,col.name=NA)

            #plot optimised site selection
            Opt_solution_pdf <- c() 
            for (ib in 1:length(cn)) {
                Opt_solution_pdf[ib] <- opt_psfs_MAC[[ib]]$value[num_steps]
            }
      
      
            Opt_solution_pdffinal <- as.data.frame(cbind(paste0("Sites"), Opt_solution_pdf))
            colnames(Opt_solution_pdffinal) <- c("Sites", "sfs_prop_m")
            Opt_Solution_Final <- as.data.frame(lapply(Opt_solution_pdffinal, unlist))
            Opt_Solution_Final$sfs_prop_m <- as.numeric(Opt_Solution_Final$sfs_prop_m)
            Opt_Solution_Final$Sites <- fct_inorder(Opt_Solution_Final$Sites)
            opt_solution_dotplot <- ggplot(Opt_Solution_Final, aes(x=Sites, y=sfs_prop_m)) + geom_dotplot(binaxis='y', binwidth=0.005, stackdir='center') + labs(y="SFS Proportion m", x="Sites")
            opt_solution_dotplot

            ### Randomisation script - generates randomised site selection to compare to optimised site selection
            rand_results <- c()
            solution_pdf <- c()
            rand_results_val <- c()

            for ( ic in 1:length(cn) ) {
                iNt <- as.numeric(cn[ic])
                for (j in 1:rand_numsteps) {
                    f <- uq-iNt
                    initial_wts <- sample(c(rep(1,iNt),rep(0, f)))
                    RandomSites <- sample(1:uq)[1:iNt]
                    rand_minw <- rep(0, uq)
                    rand_minw[RandomSites] = 1
                    rand_results[[j]] <- optimize_single_objective( gt=pmac$MAC, N_t=iNt, initial_weights=initial_wts, weights_max=max_wts, measure="psfs", max_steps=1, max_t=0.005, m=20, p_depends_delta=TRUE, pMAC_mode=TRUE, Nmat=pmac$N, weights_min=rand_minw)
                    rand_results_val[j] <- as.data.frame(rand_results[[j]]$value)    
                }
                rand_results_valfinal <- cbind((paste0("Sites")), rand_results_val)
                solution_pdf <- as.data.frame(rbind(solution_pdf, rand_results_valfinal))
            }
      
      
            colnames(solution_pdf) <- c("Sites", "sfs_prop_m")
            solution_pdf2 <- as.data.frame(lapply(solution_pdf, unlist))
            solution_pdf2$Sites <- fct_inorder(solution_pdf2$Sites)
            rand_solution_plot <- ggplot(solution_pdf2, aes(x=Sites, y=sfs_prop_m))+ geom_violin() + labs(y="SFS Proportion m", x="Sites")
            write.table(solution_pdf2, paste0(subDir, species, analysis," min samples Randomisation Table.csv"),sep=",",quote=FALSE, row.name=FALSE,col.name=TRUE)
            write.table(Opt_Solution_Final, paste0(subDir, species, analysis," Optimised min samples Table.csv"),sep=",",quote=FALSE, row.name=FALSE,col.name=TRUE)

            ### Combined plot of optimised vs randomised site selection
            combined_plot <- ggplot(solution_pdf2, aes(x=Sites, y=sfs_prop_m)) + 
            geom_violin() + 
            geom_dotplot(data=Opt_Solution_Final, binaxis='y', binwidth=0.005, stackdir='center', fill="red") + 
            labs(title=paste0("Optimised vs Random for ", iNt, " Sites"), subtitle= paste0(species," ", analysis), y="SFS Proportion (m)", x="Sites Selected (n)") + 
            theme_minimal(base_size=20) + 
            theme(axis.title.y = element_text(margin = margin(t = 0, r = 20, b = 0, l = 0)), axis.title.x = element_text(margin = margin(t = 20, r = 0, b = 0, l = 0)),  plot.title = element_text(margin = margin(t = 0, r = 0, b = 10, l = 0)), plot.subtitle = element_text(margin = margin(t = 0, r = 0, b = 20, l = 0)))    
            combined_plot
            ggsave(paste0(species, analysis," optimised min samples VS random.pdf"), path=paste0(subDir), width = 13, height = 12, dpi = 600, units = "in")  
      
            #Now take the optimised site combination and input these as forced allele proportion Sites 
            #to compare between optimised vs random allele proportion      
            solution_table_samp$row_names <- row.names(solution_table_samp)
            OrderforOptimisedSites <- solution_table_samp[order(solution_table_samp$Sites, decreasing =TRUE),]
            OptimisedSites <- OrderforOptimisedSites$row_names[1:cn]
      
            #create a temp meta dataframe to remove Sites/samples that include less than X individuals
            temp<-as.data.frame(m1$analyses)
            # Below lines remove Sites with less than threshold number of samples, and subsample Sites that are above the threshold  
            samplethreshold <-5 # chnage this to what number of samples you want toi have uniform across all pops  
        
            for (x in unique(na.omit(temp[,analysis]))) {        
                pop_samples <- which(temp[,analysis] == x)    
                if (length(pop_samples) < samplethreshold) {     
                    temp[, analysis]<-gsub(x, replacement=NA, temp[, analysis])      
                    #print(paste0("Removing ",x, " with ", " with a sample size of ",length(pop_samples)," samples"))    
                }    
                if (length(pop_samples) == samplethreshold) {      
                    #print(paste0("Keeping ",x, " with ", " with a sample size of ",length(pop_samples)," samples"))    
                }    
                if (length(pop_samples) > samplethreshold) {      
                    subsamplepops <- pop_samples[sample(1:length(pop_samples), size=samplethreshold, replace = F)]      
                    for (b in 1:length(pop_samples)){        
                        if (pop_samples[b] %in% subsamplepops == FALSE){         
                            temp[, analysis][pop_samples[b]] <- NA        
                        }        
                        #print(paste0("Reducing ",x, " to a sample size of ", samplethreshold," samples"))      
                    }    
                } else {}  
            }               
            m1$analyses<-temp      
            dm <- dart.meta.data.merge(d3, m1)      
            fields    <- c(analysis)      
            dms     <- data.by.meta.fields(dm, fields, RandRbase, species, dataset, object=analysis); #print(dms$meta$analyses[,analysis])     
            #correct inconsistencies in J's scripts for naming treatment which is for naming files
            treatment <- paste0("raw_SNPFilt_1SNPperClone_Field_", analysis)         
            dms$treatment <- treatment         
            gta <- dms$gt             
            gt_sw_comp <- gta    
            ncol(gt_sw_comp) 

            ###load required code
            source_url("https://github.com/RichardDimon/get_maf/blob/main/get_maf.r?raw=TRUE")
            source_url("https://github.com/RichardDimon/common_allele_count/blob/main/common_allele_count.r?raw=TRUE")
            ###the next line of code runs a loop that create a table with all 
            ###possible combinations of sampling assuming all included pops have atleast 5 samples available
            sampling<-setNames(data.frame(matrix(ncol = 3, nrow = 0)), c("Sites", "samples", "total"))
            for (c in 1:length(unique(dms$meta$analyses[,analysis]))) { 
                for (ind in 1:samplethreshold) {
                    sampling[nrow(sampling) + 1,] = list(c,ind,c*ind) 
                }
            }
      
            ###this bit orders the sampling strategy and removes strategies with less than x samples
      
            sampling<-sampling[order(sampling$total),]
            ###script randomly selects selected number of Sites and in each site a number of samples
            dms_meta <- cbind.data.frame(sample=dms$meta$sample_names, site=dms$meta$analyses[,analysis],lat=dms$meta$lat, long=dms$meta$long)
            colnames(dms_meta) <- c("sample","site","lat","long")
            unique_Sites <- unique(na.omit(dms_meta$site)) #maximum number of Sites
            gvals <- list() #accumulate data to be saved #samples & Sites to choose, number of common alleles

            #insert loop here to run through table listing parameters in bottom two lines#
            OGM_DF<-setNames(data.frame(matrix(ncol = 8, nrow = 0)), c("m","t_num_indv", "n_Sites_sel", "n_indiv_sel", "rand_Sites_sel", 
            "rand_indiv_sel", "jvals", "Aprop"))
            set.seed(12345)   
            ###Set Sites to select by name and set numbers of Sites and individuals to select###
            Sites_sel <- OptimisedSites
            #Sites_sel <- unique(na.omit(dms_meta$site))       
            rand_indiv_sel <- c()
            n_Sites_sel <- as.numeric(cn)
            t_num_indv <- as.numeric(dn)
            n_indiv_sel <- t_num_indv/n_Sites_sel         

            for (m in 1:1000) {
                rand_Sites_sel <- Sites_sel[sample(1:length(Sites_sel))[1:n_Sites_sel]]; #print(rand_Sites_sel)  
                rand_indiv_sel <- c()      
                for (s in 1:n_Sites_sel) {    
                    rand_indiv <- as.character(dms_meta$sample[which(dms_meta$site == rand_Sites_sel[s])][sample(1:length(which(dms_meta$site == rand_Sites_sel[s])))[1:n_indiv_sel]])     
                    rand_indiv_sel <- c(rand_indiv_sel,rand_indiv) ; #print(rand_indiv_sel)  
                }    
                # dms_meta$site[which(dms_meta$sample %in% rand_indiv_sel)]
                fixed_indi <- which(dms$sample_names %in% rand_indiv_sel)      
                ran_vec <- rep(0, nrow(gt_sw_comp))  
                ran_vec[fixed_indi] <- 1 
                common_alleles  <- common_allele_count(gt_sw_comp, ran_vec)     
                sw_maf    <- get_maf(gt_sw_comp) 
                threshold_maf <- thresh_maf #change to higher if singleton is stronger possibility e.g Cattai   
                #common allel count is 1, minor allele count observed once  
                i_sw_common <- which(sw_maf > threshold_maf)  
                jvals <- length( intersect( which(common_alleles[[2]] > 0), i_sw_common))   
                Aprop<-jvals/length(i_sw_common)       
                gvals[[m]] <- data.frame(m,t_num_indv,n_Sites_sel,n_indiv_sel,rand_Sites_sel,rand_indiv_sel,jvals, Aprop)     
            }
      
     
            #save big_data2 somewhere... 
            big_data2 = do.call(rbind, gvals)
            write.csv(big_data2, file = paste0(subDir,"forced",n_Sites_sel,"Sites_",n_indiv_sel,"individuals.csv"), row.names = FALSE) 
            OGM_DF<-rbind(OGM_DF, big_data2)
            clean_data_optimised <- OGM_DF %>% group_by(t_num_indv,n_Sites_sel,m) %>% summarize(Average_Allele=mean(Aprop),Average_Allelen=mean(jvals)) %>% data.frame
            #clean_data$t_num_indv<-as.numeric(clean_data$t_num_indv)
            write.csv(clean_data_optimised, file = paste0(subDir, "Optimised_AlleleProp_Summary.csv"), row.names = FALSE)

            #Now repreat the same using randomisation of all Sites withinthe dataset (not just the selected forced Sites)
            Sites_sel <- unique(na.omit(dms_meta$site))      
            rand_indiv_sel <- c()

            for (m in 1:1000) {
                rand_Sites_sel <- Sites_sel[sample(1:length(Sites_sel))[1:n_Sites_sel]]; #print(rand_Sites_sel)
                rand_indiv_sel <- c()      
                for (s in 1:n_Sites_sel) {   
                    rand_indiv <- as.character(dms_meta$sample[which(dms_meta$site == rand_Sites_sel[s])][sample(1:length(which(dms_meta$site == rand_Sites_sel[s])))[1:n_indiv_sel]])   
                    rand_indiv_sel <- c(rand_indiv_sel,rand_indiv) ; #print(rand_indiv_sel)  
                }    
                # dms_meta$site[which(dms_meta$sample %in% rand_indiv_sel)]   
                fixed_indi <- which(dms$sample_names %in% rand_indiv_sel)     
                ran_vec <- rep(0, nrow(gt_sw_comp))   
                ran_vec[fixed_indi] <- 1   
                common_alleles  <- common_allele_count(gt_sw_comp, ran_vec)       
                sw_maf    <- get_maf(gt_sw_comp)   
                threshold_maf <- thresh_maf #change to higher if singleton is stronger possibility e.g Cattai  
                #common allel count is 1, minor allele count observed once  
                i_sw_common <- which(sw_maf > threshold_maf)  
                jvals <- length( intersect( which(common_alleles[[2]] > 0), i_sw_common))  
                Aprop<-jvals/length(i_sw_common)       
                gvals[[m]] <- data.frame(m,t_num_indv,n_Sites_sel,n_indiv_sel,rand_Sites_sel,rand_indiv_sel,jvals, Aprop)     
            }        
            #save big_data2 somewhere...
            big_data2 = do.call(rbind, gvals)
            write.csv(big_data2, file = paste0(subDir,"randomisation_",n_Sites_sel,"Sites_",n_indiv_sel,"individuals.csv"), row.names = FALSE)
            OGM_DF<-rbind(OGM_DF, big_data2)
            clean_data_random <- OGM_DF %>% group_by(t_num_indv,n_Sites_sel,m) %>% summarize(Average_Allele=mean(Aprop),Average_Allelen=mean(jvals)) %>% data.frame
            #clean_data$t_num_indv<-as.numeric(clean_data$t_num_indv)
            write.csv(clean_data_random, file = paste0(subDir, "Random_AlleleProp_Summary.csv"), row.names = FALSE)

            
            
        }
                
        #FORCED SITE Optimisation            
        if ((ForcedOptSiteMix == 'yes') == TRUE |(ForcedOptSiteMix == 'Yes') == TRUE |(ForcedOptSiteMix == 'Y') == TRUE ){
            #reads the string variable from the parameters file and forced optimisation selection in the binary format for minw
            ForcedOptSites <- str_split(ListForcedSites, ",", simplify=TRUE);
            uqpops <- unique(pops)
            if (length(ForcedOptSites) > cn) {
                print(paste0("Warning: too many forced sites! need to supply ", cn, " or fewer sites"))
            } else { 
                for (p in 1:length(ForcedOptSites)){
                    uqpops <- gsub(ForcedOptSites[p], "1", uqpops)
                }
                for (v in 1:length(uqpops)){
                    if (uqpops[v] != 1){
                        uqpops <- gsub(uqpops[v], "0", uqpops)
                    } else {}
    
                }
                minw <- uqpops
                print("Running Forced OptSiteMix for minimum samples")   
                subDir <-paste0(outputloc2, "OptSiteMix/Minimum_Samples/ForcedSiteSelection/")
                if(!dir.exists(file.path(subDir))){
                    dir.create(file.path(subDir))
                }

                gta  <- dms$gt
                pops <- as.vector(na.omit(dms$meta$analyses[,analysis]))    
                tmpmac <- gt_to_pop_minor_allele_counts(gta, pops)    

                if (any(which(colSums(tmpmac$N < 2)>0)) == TRUE | any(which(colSums(tmpmac$MAC == 0)==nrow(tmpmac$MAC))) == TRUE) {
                    i_loc_miss <- which(colSums(tmpmac$N < 2)>0)
                    i_mac_zero <- which(colSums(tmpmac$MAC == 0)==nrow(tmpmac$MAC))
                    i_rm <- union(i_loc_miss, i_mac_zero)
                    gta <- gta[, -i_rm]  
                    pmac <- gt_to_pop_minor_allele_counts(gta, pops)
                } else {
                    pmac <- gt_to_pop_minor_allele_counts(gta, pops)
                }
                uq <- length(unique(pops)) 
                opt_psfs_MAC <- list()
                print("running forced site selection...")
            }
    
    
            for ( m in 1:length(cn) ) {
                N_t <- as.numeric(cn[m])
                cat("\n Running ", N_t, " ...\n")
                max_wts <- rep(1, uq)
                f <- uq-N_t
                initial_wts <- sample(c(rep(1,N_t),rep(0, f)))
                #N_t_MACvar <- paste0("opt_psfs_MAC_", N_t)
                opt_psfs_MAC[[m]] <- optimize_single_objective( gt=pmac$MAC, N_t=N_t, initial_weights=initial_wts, weights_max=max_wts, measure="psfs", max_steps=num_steps, max_t=0.005, m=20, p_depends_delta=TRUE, pMAC_mode=TRUE, Nmat=pmac$N, weights_min=minw)       

                png(file = paste0(subDir, species, analysis,"optpsfs_minsamples_MAC_temperature_", N_t, "_Forced_Sites.png"), width = 13.3, height = 7.5, res = 600, units = "in")
                plot(opt_psfs_MAC[[m]]$value)         
                dev.off()     


                if (length(cn) >1){   
                    ### Generate table of optimised site selection
                    solution_table_samp <- mat.or.vec(nrow(pmac$MAC), length(cn))
                    for (i in 1:length(cn)) {
                    solution_table_samp[,i] <- cbind(opt_psfs_MAC[[i]]$weight[num_steps,])        
                }

                }else{
                    solution_table_samp = NULL
                    for (i in 1:length(cn)) {   
                        solution_table_samp <- as.data.frame(cbind(opt_psfs_MAC[[i]]$weight[num_steps,]))       
                    }
                }   
      
                colnames(solution_table_samp) <- paste0("Sites")
                rownames(solution_table_samp) <- rownames(pmac$MAC)     
                write.table(solution_table_samp, paste0(subDir, species, analysis," ogmsfs min samples.csv"),sep=",",quote=FALSE, row.name=TRUE,col.name=NA)      
                #plot optimised site selection
                Opt_solution_pdf <- c() 
                for (ib in 1:length(cn)) {
                    Opt_solution_pdf[ib] <- opt_psfs_MAC[[ib]]$value[num_steps]
                }

                Opt_solution_pdffinal <- as.data.frame(cbind(paste0("Sites"), Opt_solution_pdf))
                colnames(Opt_solution_pdffinal) <- c("Sites", "sfs_prop_m")
                Opt_Solution_Final <- as.data.frame(lapply(Opt_solution_pdffinal, unlist))
                Opt_Solution_Final$sfs_prop_m <- as.numeric(Opt_Solution_Final$sfs_prop_m)
                Opt_Solution_Final$Sites <- fct_inorder(Opt_Solution_Final$Sites)
                opt_solution_dotplot <- ggplot(Opt_Solution_Final, aes(x=Sites, y=sfs_prop_m)) + geom_dotplot(binaxis='y', binwidth=0.005, stackdir='center') + labs(y="SFS Proportion m", x="Sites")
                opt_solution_dotplot

                ### Randomisation script - generates randomised site selection to compare to optimised site selection
                rand_results <- c()
                solution_pdf <- c()
                rand_results_val <- c()
                for ( ic in 1:length(cn) ) {
        
                    iNt <- as.numeric(cn[ic])
                    for (j in 1:rand_numsteps) {
                      f <- uq-iNt
                      initial_wts <- sample(c(rep(1,iNt),rep(0, f)))
                      RandomSites <- sample(1:uq)[1:iNt]
                      rand_minw <- rep(0, uq)
                      rand_minw[RandomSites] = 1
                      rand_results[[j]] <- optimize_single_objective( gt=pmac$MAC, N_t=iNt, initial_weights=initial_wts, weights_max=max_wts, measure="psfs", max_steps=1, max_t=0.005, m=20, p_depends_delta=TRUE, pMAC_mode=TRUE, Nmat=pmac$N, weights_min=rand_minw)
                      rand_results_val[j] <- as.data.frame(rand_results[[j]]$value)    
                    }
                    rand_results_valfinal <- cbind((paste0("Sites")), rand_results_val)
                    solution_pdf <- as.data.frame(rbind(solution_pdf, rand_results_valfinal))
                }
                colnames(solution_pdf) <- c("Sites", "sfs_prop_m")
                solution_pdf2 <- as.data.frame(lapply(solution_pdf, unlist))
                solution_pdf2$Sites <- fct_inorder(solution_pdf2$Sites)
                rand_solution_plot <- ggplot(solution_pdf2, aes(x=Sites, y=sfs_prop_m))+ geom_violin() + labs(y="SFS Proportion m", x="Sites")

                write.table(solution_pdf2, paste0(subDir, species, analysis," min samples Randomisation Table.csv"),sep=",",quote=FALSE, row.name=FALSE,col.name=TRUE)
                write.table(Opt_Solution_Final, paste0(subDir, species, analysis," Forced Optimised min samples Table.csv"),sep=",",quote=FALSE, row.name=FALSE,col.name=TRUE)

                ### Combined plot of optimised vs randomised site selection
                combined_plot <- ggplot(solution_pdf2, aes(x=Sites, y=sfs_prop_m)) + 
                geom_violin() + 
                geom_dotplot(data=Opt_Solution_Final, binaxis='y', binwidth=0.005, stackdir='center', fill="red") + 
                labs(title=paste0("Forced Optimised vs Random for ", iNt, " Sites"), subtitle= paste0(species," ", analysis), y="SFS Proportion (m)", x="Sites Selected (n)") + 
                theme_minimal(base_size=20) + 
                theme(axis.title.y = element_text(margin = margin(t = 0, r = 20, b = 0, l = 0)), axis.title.x = element_text(margin = margin(t = 20, r = 0, b = 0, l = 0)),  plot.title = element_text(margin = margin(t = 0, r = 0, b = 10, l = 0)), plot.subtitle = element_text(margin = margin(t = 0, r = 0, b = 20, l = 0)))    
                combined_plot
                ggsave(paste0(species, analysis," forced optimised min samples VS random.pdf"), path=paste0(subDir), width = 13, height = 12, dpi = 600, units = "in")  
      
                #Now take the optimised site combination and input these as forced allele proportion Sites 
                #to compare between optimised vs random allele proportion      
                solution_table_samp$row_names <- row.names(solution_table_samp)
                OrderforforcedOptimisedSites <- solution_table_samp[order(solution_table_samp$Sites, decreasing =TRUE),]
                forcedOptimisedSites <- OrderforforcedOptimisedSites$row_names[1:cn]

                #create a temp meta dataframe to remove Sites/samples that include less than X individuals
                temp<-as.data.frame(m1$analyses)

                # Below lines remove Sites with less than threshold number of samples, and subsample Sites that are above the threshold  
                samplethreshold <-5 # chnage this to what number of samples you want toi have uniform across all pops  
                for (x in unique(na.omit(temp[,analysis]))) {    
                    pop_samples <- which(temp[,analysis] == x)    
                    if (length(pop_samples) < samplethreshold) {     
                        temp[, analysis]<-gsub(x, replacement=NA, temp[, analysis])      
                        #print(paste0("Removing ",x, " with ", " with a sample size of ",length(pop_samples)," samples"))    
                    }    
                    if (length(pop_samples) == samplethreshold) {      
                        #print(paste0("Keeping ",x, " with ", " with a sample size of ",length(pop_samples)," samples"))    
                    }    
                    if (length(pop_samples) > samplethreshold) {      
                        subsamplepops <- pop_samples[sample(1:length(pop_samples), size=samplethreshold, replace = F)]      
                    for (b in 1:length(pop_samples)){        
                        if (pop_samples[b] %in% subsamplepops == FALSE){         
                            temp[, analysis][pop_samples[b]] <- NA        
                        }        
                        #print(paste0("Reducing ",x, " to a sample size of ", samplethreshold," samples"))      
                    }    
                    } else {}  
                }
                
        
                m1$analyses<-temp      
                dm <- dart.meta.data.merge(d3, m1)      
                fields    <- c(analysis)      
                dms     <- data.by.meta.fields(dm, fields, RandRbase, species, dataset, object=analysis); #print(dms$meta$analyses[,analysis])     
                #correct inconsistencies in J's scripts for naming treatment which is for naming files
                treatment <- paste0("raw_SNPFilt_1SNPperClone_Field_", analysis)         
                dms$treatment <- treatment         
                gta <- dms$gt             
                gt_sw_comp <- gta    
                ncol(gt_sw_comp) 

                ###load required code
                source_url("https://github.com/RichardDimon/get_maf/blob/main/get_maf.r?raw=TRUE")
                source_url("https://github.com/RichardDimon/common_allele_count/blob/main/common_allele_count.r?raw=TRUE")
                ###the next line of code runs a loop that create a table with all 
                ###possible combinations of sampling assuming all included pops have atleast 5 samples available
                sampling<-setNames(data.frame(matrix(ncol = 3, nrow = 0)), c("Sites", "samples", "total"))
                for (c in 1:length(unique(dms$meta$analyses[,analysis]))) { 
                    for (ind in 1:samplethreshold) {
                        sampling[nrow(sampling) + 1,] = list(c,ind,c*ind) 
                    }
                }
      
                ###this bit orders the sampling strategy and removes strategies with less than x samples
                sampling<-sampling[order(sampling$total),]
                ###script randomly selects selected number of Sites and in each site a number of samples
                dms_meta <- cbind.data.frame(sample=dms$meta$sample_names, site=dms$meta$analyses[,analysis],lat=dms$meta$lat, long=dms$meta$long)
                colnames(dms_meta) <- c("sample","site","lat","long")
                unique_Sites <- unique(na.omit(dms_meta$site)) #maximum number of Sites
                gvals <- list() #accumulate data to be saved #samples & Sites to choose, number of common alleles

                #insert loop here to run through table listing parameters in bottom two lines#
                OGM_DF<-setNames(data.frame(matrix(ncol = 8, nrow = 0)), c("m","t_num_indv", "n_Sites_sel", "n_indiv_sel", "rand_Sites_sel", 
                "rand_indiv_sel", "jvals", "Aprop"))
                set.seed(12345)   
                ###Set Sites to select by name and set numbers of Sites and individuals to select###
                Sites_sel <- forcedOptimisedSites
                #Sites_sel <- unique(na.omit(dms_meta$site))       
                rand_indiv_sel <- c()
                n_Sites_sel <- as.numeric(cn)
                t_num_indv <- as.numeric(dn)
                n_indiv_sel <- t_num_indv/n_Sites_sel         

                for (m in 1:1000) {
                    rand_Sites_sel <- Sites_sel[sample(1:length(Sites_sel))[1:n_Sites_sel]]; #print(rand_Sites_sel)  
                    rand_indiv_sel <- c()      
                    for (s in 1:n_Sites_sel) {    
                        rand_indiv <- as.character(dms_meta$sample[which(dms_meta$site == rand_Sites_sel[s])][sample(1:length(which(dms_meta$site == rand_Sites_sel[s])))[1:n_indiv_sel]])     
                        rand_indiv_sel <- c(rand_indiv_sel,rand_indiv) ; #print(rand_indiv_sel)  
                    }    
        
                    # dms_meta$site[which(dms_meta$sample %in% rand_indiv_sel)]
                    fixed_indi <- which(dms$sample_names %in% rand_indiv_sel)      
                    ran_vec <- rep(0, nrow(gt_sw_comp))  
                    ran_vec[fixed_indi] <- 1 
                    common_alleles  <- common_allele_count(gt_sw_comp, ran_vec)     
                    sw_maf    <- get_maf(gt_sw_comp) 
                    threshold_maf <- thresh_maf #change to higher if singleton is stronger possibility e.g Cattai   
                    #common allel count is 1, minor allele count observed once  
                    i_sw_common <- which(sw_maf > threshold_maf)  
                    jvals <- length( intersect( which(common_alleles[[2]] > 0), i_sw_common))   
                    Aprop<-jvals/length(i_sw_common)       
                    gvals[[m]] <- data.frame(m,t_num_indv,n_Sites_sel,n_indiv_sel,rand_Sites_sel,rand_indiv_sel,jvals, Aprop)     
                }
      
                #save big_data2 somewhere... 
                big_data2 = do.call(rbind, gvals)
                write.csv(big_data2, file = paste0(subDir,"forced",n_Sites_sel,"Sites_",n_indiv_sel,"individuals.csv"), row.names = FALSE) 
                OGM_DF<-rbind(OGM_DF, big_data2)
                clean_data_forcedoptimised <- OGM_DF %>% group_by(t_num_indv,n_Sites_sel,m) %>% summarize(Average_Allele=mean(Aprop),Average_Allelen=mean(jvals)) %>% data.frame
                #clean_data$t_num_indv<-as.numeric(clean_data$t_num_indv)
                write.csv(clean_data_forcedoptimised, file = paste0(subDir, "Forced_Optimisation_AlleleProp_Summary.csv"), row.names = FALSE)

                #now combine all results into a summmary csv file (with forced optimisation)                                         
                final_output <- cbind.data.frame(analysis, "Minimum Samples", samplingthreshold, cn, dn/cn, dn, paste0(OptimisedSites, collapse=', '), clean_data_optimised$Average_Allele, clean_data_optimised$Average_Allelen, ListForcedSites, paste0(forcedOptimisedSites, collapse=', '), clean_data_forcedoptimised$Average_Allele, clean_data_forcedoptimised$Average_Allelen, clean_data_random$Average_Allele, clean_data_random$Average_Allelen)     
                colnames(final_output) <- c("Analysis", "Selection", "Sample Threshold", "N Sites",  "N Samples per Site", "N Total Samples", "Optimised Sites", "Optimised Allele Prop", "Optimised N Alleles", "Intial Forced Sites", "Forced Optimised Sites", "Forced Optimised Allele Prop", "Forced Optimised N Alleles", "Random Allele Prop", "Random N Alleles")        
                subDir <-paste0(outputloc2, "OptSiteMix/")
                write.csv(final_output, file = paste0(subDir, "Forced_OptSiteMix_MinSamples_Final_Output.csv"), row.names= FALSE)   
                
            }

        } else {

            #now combine all results into a summmary csv file (without forced optimisation)                                          
            final_output <- cbind.data.frame(analysis, "Minimum Samples", samplingthreshold, cn, dn/cn, dn, paste0(OptimisedSites, collapse=', '), clean_data_optimised$Average_Allele, clean_data_optimised$Average_Allelen, clean_data_random$Average_Allele, clean_data_random$Average_Allelen)     
            colnames(final_output) <- c("Analysis", "Selection", "Sample Threshold", "N Sites",  "N Samples per Site", "N Total Samples", "Optimised Sites", "Optimised Allele Prop", "Optimised N Alleles", "Random Allele Prop", "Random N Alleles")        
            subDir <-paste0(outputloc2, "OptSiteMix/")
            write.csv(final_output, file = paste0(subDir, "OptSiteMix_MinSamples_Final_Output.csv"), row.names= FALSE)   
        }
    } else {
        print("not running OptSiteMix for minimum samples")    
    }
    
#}




[1] "Starting ArgyTrif"

 Reading data file: ArgyTrif/dart_raw/Report_DArg23-8132_SNP_singlerow.csv 
 Initial data scan -- 
   Samples:  424  
   SNPs:  17627  
   Loci:  15520  

 Creating a DArT data list containing:                       
             Genotypes                    --  $gt            
             Sample Names                 --  $sample_names  
             Locus Names                  --  $locus_names   
             Locus Reproducibility Scores --  $locus_repro   
             Position of SNP in locus     --  $locus_pos     
             Data filtering treatments    --  $treatment     
             Position of SNP in locus     --  $locus_pos     
             Method of data encoding gt   --  $encoding      
             Nucleotides in this SNP      --  $locus_nuc     

 DArT object found with dart encoding (0=hom ref; 1=hom alt; 2=het) 
 Rewriting with altcount encoding     (0=hom ref; 1=het; 2=hom alt) 
  QC directory:  ArgyTrif/qual_stat/raw  already exists, cont

[1] "notes"                         "NurseryPopsKinship"           
 [3] "BigScrubRegionPopsNoNursery"   "BigScrubRegionPopsWithNursery"
 [5] "AllPopsNoNursery"              "AllPopsWithNursery"           
 [7] "Clades"                        "SouthernPops"                 
 [9] "nationalPark"                  "natureReserve"

[1] "Removing Bilborough Springbrook with  with a sample size of 3 samples"
[1] "Removing Blackbutt with  with a sample size of 4 samples"
[1] "Removing ANBG ex-situ Mt Nardi with  with a sample size of 1 samples"
 Data found in altcount encoding, search for fixed sites commencing 
 Found  640  loci with MAC lte  0  returning their indices 
 Found  640 fixed SNPs, these will be removed from DArT object 
 Samples successfully matched and ordered: merging dart and meta objects 
 Data found in altcount encoding, search for fixed sites commencing 
 Found  172  loci with MAC lte  0  returning their indices 
 Found  172 fixed SNPs, these will be removed from DArT object 
 Samples successfully matched and ordered: merging dart and meta objects 
  Dart standard directory:  ArgyTrif/dart_standard  already exists... content might be overwritten. 
  Standard directory:  ArgyTrif/dart_standard/raw_SNPFilt_1SNPperClone_Field_AllPopsNoNursery  already exists, content will be overwritten. 


[1] 2208

`summarise()` has grouped output by 't_num_indv', 'n_Sites_sel', 'combined'. You can override using the `.groups` argument.
Scale for fill is already present.
Adding another scale for fill, which will replace the existing scale.


[1] "AlleleProp analysis for ArgyTrif AllPopsNoNursery complete"
[1] "not running OptSiteMix for minimum Sites"
[1] "not running OptSiteMix for minimum samples"
